In [1]:
from retrievalAugmentedTransformer import RAT, Data_Processing

/Users/dipalshah/Desktop/bitsdatathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name ProsusAI/finbert. Creating a new one with mean pooling.


In [2]:
rat = RAT("AAPL", 22)
dp = Data_Processing(rat)
df = dp.get_data("AAPL", "2020-01-01", "2025-04-02")

X, y = dp.format_data_combined(df, 10, 3)
print(X)
print(y)

/Users/dipalshah/Desktop/bitsdatathon/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  13 of 13 completed


tensor([[[-1.8734,  0.8852, -0.4331,  ..., -1.6952, -1.4825, -1.0864],
         [-1.8894,  1.0909, -0.4331,  ..., -1.7427, -1.5193, -0.8954],
         [-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         ...,
         [-1.7828,  0.6206, -0.4331,  ..., -1.6786, -1.4883, -1.1049],
         [-1.8062,  1.3874, -0.4331,  ..., -1.6456, -1.4896, -1.0962],
         [-1.8135,  0.6280, -0.4331,  ..., -1.5981, -1.4995, -1.0925]],

        [[-1.8894,  1.0909, -0.4331,  ..., -1.7427, -1.5193, -0.8954],
         [-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         [-1.8841,  0.3804, -0.4331,  ..., -1.7197, -1.5142, -0.9237],
         ...,
         [-1.8062,  1.3874, -0.4331,  ..., -1.6456, -1.4896, -1.0962],
         [-1.8135,  0.6280, -0.4331,  ..., -1.5981, -1.4995, -1.0925],
         [-1.7922,  0.3796, -0.4331,  ..., -1.5692, -1.4610, -1.1049]],

        [[-1.8764,  0.5609, -0.4331,  ..., -1.7091, -1.5074, -0.9163],
         [-1.8841,  0.3804, -0.4331,  ..., -1

In [6]:
rat.train_model(X, y, epochs = 1000)

/Users/dipalshah/Desktop/bitsdatathon/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/1000, Train Loss: 0.762115, Val Loss: 3.702325
Epoch 5/1000, Train Loss: 0.762586, Val Loss: 3.747959
Epoch 10/1000, Train Loss: 0.724932, Val Loss: 3.654082
Epoch 15/1000, Train Loss: 0.723927, Val Loss: 3.633657
Epoch 20/1000, Train Loss: 0.722986, Val Loss: 3.611649
Epoch 25/1000, Train Loss: 0.719913, Val Loss: 3.616840
Epoch 30/1000, Train Loss: 0.704524, Val Loss: 3.592285
Epoch 35/1000, Train Loss: 0.704690, Val Loss: 3.579229
Epoch 40/1000, Train Loss: 0.704698, Val Loss: 3.574420
Epoch 45/1000, Train Loss: 0.704668, Val Loss: 3.572616
Epoch 50/1000, Train Loss: 0.704616, Val Loss: 3.571881
Epoch 55/1000, Train Loss: 0.704579, Val Loss: 3.571541
Epoch 60/1000, Train Loss: 0.704540, Val Loss: 3.571340
Epoch 65/1000, Train Loss: 0.697173, Val Loss: 3.564851
Epoch 70/1000, Train Loss: 0.697322, Val Loss: 3.560666
Epoch 75/1000, Train Loss: 0.697422, Val Loss: 3.558034
Epoch 80/1000, Train Loss: 0.697484, Val Loss: 3.556394
Epoch 85/1000, Train Loss: 0.697519, Val Loss: 3.5

KeyboardInterrupt: 

In [8]:
import torch
torch.save(rat.state_dict(), "saved_models/rat.pth")

In [4]:
import pandas as pd
import requests

def fetch_macro_data():
    # FRED series IDs for macroeconomic indicators
    series_ids = {
        "Fed_Funds_Rate": "FEDFUNDS",
        "CPI": "CPIAUCSL",
        "GDP": "GDPC1",
        "Unemployment_Rate": "UNRATE",
        "Money_Supply": "M2SL",
        "Retail_Sales": "RSXFS",
        "Oil_Price": "IR14250"
    }

    base_url = "https://api.stlouisfed.org/fred/series/observations"
    macro_data = pd.DataFrame()

    # Get the full range of dates you need (e.g., from 2000-01-01 to the most recent available date)
    date_range = pd.date_range(start="2000-01-01", end=pd.to_datetime('today'))

    for column, series_id in series_ids.items():
        # Construct the URL and parameters
        params = {
            "api_key": "d2a5f8b122ea6046576eb67cd258404f",  # Replace with your actual API key
            "file_type": "json",
            "series_id": series_id,
            "observation_start": "2000-01-01",
        }

        # Fetch the data
        response = requests.get(base_url, params=params)
        data = response.json()

        # Parse and format the response data
        observations = data.get("observations", [])
        df = pd.DataFrame(observations)
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        df["value"] = pd.to_numeric(df["value"], errors="coerce")

        # Create a DataFrame with all the dates in the date range
        full_df = pd.DataFrame(index=date_range)

        # Merge the fetched data with the full date range
        full_df = full_df.merge(df["value"], how="left", left_index=True, right_index=True)
        
        # Forward fill missing data
        full_df["value"] = full_df["value"].ffill()

        # Add the data to the macro_data DataFrame
        macro_data[column] = full_df["value"]

    # Return the final DataFrame with all macroeconomic indicators
    return macro_data

# Example usage:
macro_data = fetch_macro_data()
#macro_data.to_csv("../data/new_macro_data.csv", index = True)